## Dependencies

In [ ]:
%pip install --quiet -U langchain langchain-community pymupdf langchain-huggingface langchain-pinecone pinecone-notebooks tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 73.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.9/438.9 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.3/524.3 kB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB

## Environment

In [ ]:
from google.colab import userdata
import os

In [ ]:
os.environ['PINECONE_API_KEY']=userdata.get('PINECONE_API_KEY')

In [ ]:
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone,ServerlessSpec


from pathlib import Path
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings

from uuid import uuid4
import time

## Document Loading and Chunking

In [ ]:
data_dir = '/content/Knowledge_base'
[print(i) for i in Path(data_dir).glob("*.pdf")]

/content/Knowledge_base/Transportation.pdf
/content/Knowledge_base/Infra_corruption_municipal.pdf
/content/Knowledge_base/Cybercrime.pdf
/content/Knowledge_base/Miscellaneous.pdf


[None, None, None, None]

In [ ]:
# import re
from pathlib import Path
from pymupdf import Document
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

def process_pdf_document(file_path: str) -> list:
    """Process a single PDF document with enhanced metadata"""
    loader = PyMuPDFLoader(file_path)
    docs = loader.load()
    return docs

def preprocess_legal_documents(data_dir: str = '/content/Knowledge_base') -> list:
    """Process all PDFs in directory with optimal chunking"""
    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=1000,
        chunk_overlap=200,
    )

    all_docs = []
    for pdf_file in Path(data_dir).glob('*.pdf'):
        print(f"Processing {pdf_file.name}...")
        docs = process_pdf_document(str(pdf_file))
        chunks = text_splitter.split_documents(docs)
        all_docs.extend(chunks)

    print(f"Processed {len(all_docs)} document chunks")
    return all_docs

# Usage
documents = preprocess_legal_documents()

Processing Transportation.pdf...
Processing Infra_corruption_municipal.pdf...
Processing Cybercrime.pdf...
Processing Miscellaneous.pdf...
Processed 149 document chunks


In [ ]:
documents[:1]

[Document(metadata={'producer': 'Skia/PDF m127', 'creator': 'Chromium', 'creationdate': '2025-06-25T07:43:09+00:00', 'source': '/content/Knowledge_base/Transportation.pdf', 'file_path': '/content/Knowledge_base/Transportation.pdf', 'total_pages': 62, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2025-06-25T07:43:09+00:00', 'trapped': '', 'modDate': "D:20250625074309+00'00'", 'creationDate': "D:20250625074309+00'00'", 'page': 0}, page_content='This knowledge database serves as a complete reference guide for Indian citizens navigating\ntransportation-related legal matters. It covers all modes of transport including railways, road\nvehicles, aviation, shipping, and metro systems. The information is structured for easy\nintegration into RAG-based chatbot systems.\nPrimary Question (English): What are the cancellation charges and refund rules for railway\ntickets in India?\nHindi Translation: भारत में रेलवे टिकट रद्द करने के शुल्क और रिफंड नियम क

## Embedding & Vector Store

In [ ]:
index_name = 'nyayasetu'
pc = Pinecone() #setting up connection with PineCone

In [ ]:
existing_index = [i.name for i in pc.list_indexes()] #checking for existing indexes
existing_index

['advanceragpipeline', 'graminsaathi', 'krishiarya']

In [ ]:
if index_name not in existing_index:
    pc.create_index(
        name = index_name,
        dimension = 768,
        metric = 'cosine',
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)
else:
    print(f'Index {index_name} already exists')

In [ ]:
embeddings = HuggingFaceEmbeddings(
        model_name="BAAI/bge-base-en-v1.5",
        model_kwargs={'device': 'cpu'},
        encode_kwargs={'normalize_embeddings': True}
    )

index = pc.Index(index_name)
vector_store = PineconeVectorStore(index=index,embedding=embeddings)

#testing first
# single = documents[:2]
# vector_store.add_documents(documents=single)

In [ ]:
import uuid
batch_size = 100
uuids = [str(uuid.uuid4()) for _ in documents]
for i in range(0, len(documents), batch_size):
    batch_docs = documents[i:i+batch_size]
    batch_uuids = uuids[i:i+batch_size]

    try:
        vector_store.add_documents(documents=batch_docs, ids=batch_uuids)
        print(f"Batch {i//batch_size + 1} added successfully.")
    except Exception as e:
        print(f"Error adding batch {i//batch_size + 1}: {e}")


Batch 1 added successfully.
Batch 2 added successfully.


In [ ]:
#Checking Retrival

retriever = vector_store.as_retriever(
    search_type = 'similarity_score_threshold',
    search_kwargs = {'k':3,'score_threshold':0.7},
)

In [ ]:
retriever.get_relevant_documents('Fines in Railway')

/tmp/ipython-input-14-4186765475.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents('Fines in Railway')


[Document(id='d35434bd-7f69-4f69-9785-a5bb17e2508c', metadata={'author': '', 'creationDate': "D:20250625074309+00'00'", 'creationdate': '2025-06-25T07:43:09+00:00', 'creator': 'Chromium', 'file_path': '/content/Knowledge_base/Transportation.pdf', 'format': 'PDF 1.4', 'keywords': '', 'modDate': "D:20250625074309+00'00'", 'moddate': '2025-06-25T07:43:09+00:00', 'page': 19.0, 'producer': 'Skia/PDF m127', 'source': '/content/Knowledge_base/Transportation.pdf', 'subject': '', 'title': '', 'total_pages': 62.0, 'trapped': ''}, page_content='Accessibility Features:\nPenalties and Fines:\nGrievance Redressal:\nSecurity Measures:\nCommercial Activities Regulation:\nC. Legal Framework:\nPrimary Law: Metro Railway (Operations and Maintenance) Act, 2002\nFollow platform markings\nEmergency communication devices available\nWheelchair accessible stations\nBraille signage for visually impaired\nAudio announcements in multiple languages\nDesignated coaches for women\nElevator/escalator facilities\nTick